In [1]:
#  from google.colab import drive
#  drive.mount('/content/drive')

In [2]:
# !unzip /content/drive/MyDrive/archive.zip

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [4]:
train=ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)

In [5]:
test=ImageDataGenerator(rescale=1./255)

In [6]:
traindata=train.flow_from_directory('/content/tomato/train',target_size=(128,128),batch_size=16,class_mode='categorical')

Found 10000 images belonging to 10 classes.


In [7]:
testdata=test.flow_from_directory('/content/tomato/val',target_size=(128,128),batch_size=8,class_mode='categorical')

Found 1000 images belonging to 10 classes.


In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation, Flatten, Dropout, Dense

In [14]:
model = Sequential()

model.add(Conv2D(128, (3, 3),input_shape=(128,128,3)))
model.add(Activation("relu"))

model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.25))
model.add(Conv2D(128, (3, 3)))
model.add(Activation("relu"))

model.add(Conv2D(128, (3, 3)))
model.add(Activation("relu"))

model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(128, (3, 3)))
model.add(Activation("relu"))

model.add(Conv2D(128, (3, 3)))
model.add(Activation("relu"))

model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(554))
model.add(Activation("relu"))

model.add(Dropout(0.5))
model.add(Dense(10))
model.add(Activation("softmax"))

Batch normalization (also known as batch norm) is a method used to make training of artificial neural networks faster and more stable through normalization of the layers' inputs by re-centering and re-scaling.

DROPOUT GIVES NEURONS FROM ABOVE LAYER TO BELOW LAYER

In [15]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_10 (Conv2D)          (None, 126, 126, 128)     3584      
                                                                 
 activation_12 (Activation)  (None, 126, 126, 128)     0         
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 42, 42, 128)      0         
 2D)                                                             
                                                                 
 dropout_7 (Dropout)         (None, 42, 42, 128)       0         
                                                                 
 conv2d_11 (Conv2D)          (None, 40, 40, 128)       147584    
                                                                 
 activation_13 (Activation)  (None, 40, 40, 128)       0         
                                                      

In [16]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [17]:
model.fit(traindata,validation_data=traindata,epochs=5)

Epoch 1/5
625/625 [==============================] - 113s 177ms/step - loss: 1.9430 - accuracy: 0.2832 - val_loss: 1.5888 - val_accuracy: 0.4141
Epoch 2/5
625/625 [==============================] - 111s 178ms/step - loss: 1.4132 - accuracy: 0.4901 - val_loss: 1.3248 - val_accuracy: 0.5287
Epoch 3/5
625/625 [==============================] - 113s 181ms/step - loss: 1.2142 - accuracy: 0.5633 - val_loss: 0.9562 - val_accuracy: 0.6527
Epoch 4/5
625/625 [==============================] - 112s 179ms/step - loss: 1.0627 - accuracy: 0.6125 - val_loss: 0.8459 - val_accuracy: 0.6888
Epoch 5/5
625/625 [==============================] - 113s 181ms/step - loss: 0.9120 - accuracy: 0.6674 - val_loss: 0.7536 - val_accuracy: 0.7313


In [18]:
model.evaluate(traindata, verbose=1)

625/625 [==============================] - 56s 89ms/step - loss: 0.7637 - accuracy: 0.7229


[0.7637004256248474, 0.7228999733924866]

In [19]:
# model.save_weights("model_weights.hdf5")
model.save("tomato.hdf5")

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
t=cv2.imread('/content/tomato/val/Tomato___healthy/0cfee8b1-5de4-4118-9e25-f9c37bb6f17e___GH_HL Leaf 252.JPG')
cv2_imshow(t)
testimg=cv2.resize(t,(128,128))
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array
testimg=img_to_array(testimg)/255
h=np.expand_dims(testimg,axis=0)
r=model.predict(h)
ypred=np.argmax(r,axis=1)
ypred